#### https://github.com/vivva/DLinear
#### https://github.com/cure-lab/LTSF-Linear
##### https://github.com/bnsreenu/python_for_microscopists/blob/master/166b-Intro_to_time_series_Forecasting_using_LSTM_and_TimeseriesGenerator.py

In [12]:
# from DLinear import Model
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import numpy as np
import time

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

In [2]:
class StandardScaler():
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def transform(self, data):
        return (data - self.mean) / self.std

    def inverse_transform(self, data):
        return (data * self.std) + self.mean
    
# Custom dataset
class CSVDataset(Dataset):
    def __init__(self, path, seq_len=1):
        # Normalization is optional but recommended for neural network as certain 
        # activation functions are sensitive to magnitude of numbers. 

        self.scaler = MinMaxScaler(feature_range=(0, 1)) #Also try QuantileTransformer
        
        self.data = self.__read_data__(path)
        self.seq_len = seq_len
       
    def __read_data__(self, path):
        df = pd.read_csv(path)
        df['Month'] = pd.to_datetime(df['Month'])
        df.set_index('Month', inplace=True) 
        df.rename(columns = {'#Passengers': 'Passengers'}, inplace = True)
        # Fit the scaler to the DataFrame columns and transform
        # normalize the dataset
        df['Passengers'] = self.scaler.fit_transform(df[['Passengers']])
        # print(df.head())
        return df
    
    def __len__(self):
        return len(self.data)-self.seq_len - 1

    def __getitem__(self, idx):
        # print(type(self.data.iloc[idx + self.seq_len]['Passengers']))
        # print(type(self.data.iloc[idx:(idx + self.seq_len)]['Passengers']))
        features = torch.tensor((self.data.iloc[idx:(idx + self.seq_len)]['Passengers'].values), dtype=torch.float32)
        label = torch.tensor((self.data.iloc[idx + self.seq_len]['Passengers']), dtype=torch.float32)  # the return value is numpy float
        features = features.unsqueeze(1)
        label = label.unsqueeze(0)
        label = label.unsqueeze(0)
        return features, label

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

In [51]:
class Configs:
    def __init__(self):
        self.seq_len = 4
        self.pred_len = 1
        self.individual = False # multiplevariable or univariable
        self.enc_in = 1          # variable numbers (univariable then 1)
        self.learning_rate = 0.001
        
configs = Configs()
print(configs.seq_len)

class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x


class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

class DLinearModel(nn.Module):
    """
    DLinear
    """
    def __init__(self, configs):
        super(DLinearModel, self).__init__()
        self.seq_len = configs.seq_len
        self.pred_len = configs.pred_len

        # Decompsition Kernel Size
        kernel_size = 5
        self.decompsition = series_decomp(kernel_size)
        self.individual = configs.individual
        self.channels = configs.enc_in

        if self.individual:
            self.Linear_Seasonal = nn.ModuleList()
            self.Linear_Trend = nn.ModuleList()
            self.Linear_Decoder = nn.ModuleList()
            for i in range(self.channels):
                self.Linear_Seasonal.append(nn.Linear(self.seq_len,self.pred_len))
                self.Linear_Seasonal[i].weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
                self.Linear_Trend.append(nn.Linear(self.seq_len,self.pred_len))
                self.Linear_Trend[i].weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
                self.Linear_Decoder.append(nn.Linear(self.seq_len,self.pred_len))
        else:
            self.Linear_Seasonal = nn.Linear(self.seq_len,self.pred_len)
            self.Linear_Trend = nn.Linear(self.seq_len,self.pred_len)
            self.Linear_Decoder = nn.Linear(self.seq_len,self.pred_len)
            self.Linear_Seasonal.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))
            self.Linear_Trend.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))

    def forward(self, x):
        # x: [Batch, Input length, Channel]
        seasonal_init, trend_init = self.decompsition(x)
        seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1)
        if self.individual:
            seasonal_output = torch.zeros([seasonal_init.size(0),seasonal_init.size(1),self.pred_len],dtype=seasonal_init.dtype).to(seasonal_init.device)
            trend_output = torch.zeros([trend_init.size(0),trend_init.size(1),self.pred_len],dtype=trend_init.dtype).to(trend_init.device)
            for i in range(self.channels):
                seasonal_output[:,i,:] = self.Linear_Seasonal[i](seasonal_init[:,i,:])
                trend_output[:,i,:] = self.Linear_Trend[i](trend_init[:,i,:])
        else:
            seasonal_output = self.Linear_Seasonal(seasonal_init)
            trend_output = self.Linear_Trend(trend_init)

        x = seasonal_output + trend_output
        return x.permute(0,2,1) # to [Batch, Output length, Channel]


4


In [52]:
        
### need to work on.....................................
model = DLinearModel(configs).float()
optimizer = optim.Adam(model.parameters(), lr=configs.learning_rate)
criterion = nn.MSELoss()
# Create dataset
dataset = CSVDataset("archive/AirPassengers.csv", seq_len=4)
# Create data loader
dataloader = DataLoader(dataset, batch_size=4, shuffle=False)

In [53]:
# Get one batch of data
dataiter = iter(dataloader)
inputs, labels = next(dataiter)
print(inputs.shape)
# kernel_size = 5
# stride = 1
# avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)
# x = inputs
# print(x.shape)
# front = x[:, 0:1, :].repeat(1, (kernel_size - 1) // 2, 1)
# print(f'front shape {front.shape}')

# end = x[:, -1:, :].repeat(1, (kernel_size - 1) // 2, 1)
# x = torch.cat([front, x, end], dim=1)
# print(x.shape)
# move_mean = avg(x.permute(0, 2, 1))
# print(move_mean)
outputs = model(inputs)
print(labels)
print(outputs)
loss = criterion(outputs, labels)
print(loss)

torch.Size([4, 4, 1])
tensor([[[0.0328]],

        [[0.0598]],

        [[0.0849]],

        [[0.0849]]])
tensor([[[0.2741]],

        [[0.2784]],

        [[0.2866]],

        [[0.2943]]], grad_fn=<PermuteBackward0>)
tensor(0.0476, grad_fn=<MseLossBackward0>)


In [55]:
# Assuming you have a DataLoader `dataloader`, a model `model`, a loss function `criterion`, and an optimizer `optimizer`
num_epochs = 100
time_now = time.time()
train_steps = len(dataloader)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    iter_count = 0
    model.train()
    
    for i, data in enumerate(dataloader, 0):
        iter_count += 1
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # print(inputs.dtype, labels.dtype)
        # print(inputs.shape, labels.shape)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5 == 0:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5))
            running_loss = 0.0
        # if (i + 1) % 20 == 0:
        #     print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
        #     speed = (time.time() - time_now) / iter_count
        #     left_time = speed * ((num_epochs - epoch) * train_steps - i)
        #     print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
        #     iter_count = 0
        #     time_now = time.time()

print('Finished Training')


[1,     1] loss: 0.000
[1,     6] loss: 0.005
[1,    11] loss: 0.003
[1,    16] loss: 0.003
[1,    21] loss: 0.005
[1,    26] loss: 0.011
[1,    31] loss: 0.017
[2,     1] loss: 0.000
[2,     6] loss: 0.004
[2,    11] loss: 0.003
[2,    16] loss: 0.003
[2,    21] loss: 0.005
[2,    26] loss: 0.011
[2,    31] loss: 0.017
[3,     1] loss: 0.000
[3,     6] loss: 0.004
[3,    11] loss: 0.003
[3,    16] loss: 0.003
[3,    21] loss: 0.005
[3,    26] loss: 0.011
[3,    31] loss: 0.016
[4,     1] loss: 0.000
[4,     6] loss: 0.004
[4,    11] loss: 0.003
[4,    16] loss: 0.003
[4,    21] loss: 0.005
[4,    26] loss: 0.010
[4,    31] loss: 0.016
[5,     1] loss: 0.000
[5,     6] loss: 0.004
[5,    11] loss: 0.003
[5,    16] loss: 0.003
[5,    21] loss: 0.005
[5,    26] loss: 0.010
[5,    31] loss: 0.016
[6,     1] loss: 0.000
[6,     6] loss: 0.003
[6,    11] loss: 0.002
[6,    16] loss: 0.003
[6,    21] loss: 0.005
[6,    26] loss: 0.010
[6,    31] loss: 0.015
[7,     1] loss: 0.000
[7,     6] 

In [ ]:
# Create dataset
dataset = CSVDataset("archive/AirPassengers.csv", seq_len=4)
# Create data loader
dataloader = DataLoader(dataset, batch_size=4, shuffle=False)

# # Iterate over data
# for i, data in enumerate(dataloader):
#     print(data)

# Get one batch of data
dataiter = iter(dataloader)
inputs, labels = next(dataiter)
print(inputs.shape)
# front = inputs[:, 0:1, :].repeat(1, (3 - 1) // 2, 1)
# print(front)

# kernel_size = 1
# decompsition = series_decomp(kernel_size)
# seasonal_init, trend_init = decompsition(inputs)
# seasonal_init, trend_init = seasonal_init.permute(0,2,1), trend_init.permute(0,2,1)
# print((seasonal_init.shape))
# seq_len = 4
# pred_len = 1
# Linear_Seasonal = nn.Linear(seq_len, pred_len)

# Linear_Seasonal.weight = nn.Parameter((1/seq_len)*torch.ones([pred_len,seq_len]))
# print(Linear_Seasonal.weight.dtype)

# seasonal_output = Linear_Seasonal(seasonal_init.float())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('dark_background')


In [ ]:
# load the dataset
df = pd.read_csv('archive/AirPassengers.csv')
print(df.dtypes)

df['Month'] = pd.to_datetime(df['Month'])
print(df.dtypes)
df.set_index('Month', inplace=True) 

df.rename(columns = {'#Passengers': 'Passengers'}, inplace = True)
# print(df.head())
plt.plot(df['Passengers'])

In [10]:
for i, (batch_x, batch_y) in enumerate(dataloader):
    print(batch_x)
    print(batch_y)
    print(i)



tensor([[[0.0154],
         [0.0270],
         [0.0541],
         [0.0483]],

        [[0.0270],
         [0.0541],
         [0.0483],
         [0.0328]],

        [[0.0541],
         [0.0483],
         [0.0328],
         [0.0598]],

        [[0.0483],
         [0.0328],
         [0.0598],
         [0.0849]]])
tensor([[[0.0328]],

        [[0.0598]],

        [[0.0849]],

        [[0.0849]]])
0
tensor([[[0.0328],
         [0.0598],
         [0.0849],
         [0.0849]],

        [[0.0598],
         [0.0849],
         [0.0849],
         [0.0618]],

        [[0.0849],
         [0.0849],
         [0.0618],
         [0.0290]],

        [[0.0849],
         [0.0618],
         [0.0290],
         [0.0000]]])
tensor([[[0.0618]],

        [[0.0290]],

        [[0.0000]],

        [[0.0270]]])
1
tensor([[[0.0618],
         [0.0290],
         [0.0000],
         [0.0270]],

        [[0.0290],
         [0.0000],
         [0.0270],
         [0.0212]],

        [[0.0000],
         [0.0270],
         [